# Documentation 
https://github.com/innovata/DSE-REST-API  

# [1] 3DEXPERIENCE Platform CLM Agent 생성  
- "F:\pypjts\DSE-REST-API\How to use APIs\credentials"경로에 Agent ID, Password 저장 

# [2] 파이썬 가상환경(Python Virtual Environment) 생성 및 dserestapi 패키지 설치



In [1]:
# 개발용 경로설정
import os, sys 
sys.path.append(r"F:\pypjts\DSE-REST-API\src")

In [2]:
# 커맨드 프롬프트 사용 권장
# !pip install dserestapi

# **********************************************************************

# [3.1] 환경셋업 | Normal Case: 일반유저 (Member) 계정으로 만든 에이전트 사용

In [3]:
import os
import re 
os.environ["CLM_AGENT_CREDENTIAL_PATH"] = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\Member\CML Agent ID-PW.json"
os.environ["3DX_PLATFORM_TENANT_URI"] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"
for k, v in os.environ.items():
    if re.search(r"^CLM|^3DX", k):
        print([k, v])

['CLM_AGENT_CREDENTIAL_PATH', 'F:\\pypjts\\DSE-REST-API\\How to use APIs\\credentials\\Member\\CML Agent ID-PW.json']
['3DX_PLATFORM_TENANT_URI', 'https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443']


In [4]:
import dserestapi 
dserestapi

<module 'dserestapi' from 'F:\\pypjts\\DSE-REST-API\\src\\dserestapi\\__init__.py'>

# [3.2] 환경셋업 | Error Case: 어드민 계정으로 만든 에이전트 사용 

In [3]:
import os
import re 
os.environ["CLM_AGENT_CREDENTIAL_PATH"] = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\Admin\CML Agent ID-PW.json"
os.environ["3DX_PLATFORM_TENANT_URI"] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"
from dserestapi import Storages
api = Storages()
res = api.get()
res.json() 

{'error': 'admin role is forbidden'}

# **********************************************************************

# [4] 사용 예제 | Storages

### [4.1] 스토리지 목록 가져오기 

In [6]:
# 모든 스토리지 리스트 가져오기
from dserestapi import Storages
api = Storages()
res = api.get()
res

<Response [200]>

In [7]:
import json 
file = os.path.join(os.getcwd(), "01_Storages", "Storages List.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [ ]:
# !pip install pandas 

In [ ]:
import pandas as pd 
df = pd.DataFrame(res.json()['cards'])
df#.head(5)

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
workspace_ids = df.workspaceId.unique()
import pprint 
pp = pprint.PrettyPrinter(indent=2)
print("Unique Workspace IDs Count:", len(workspace_ids))
pp.pprint(sorted(workspace_ids))

### [4.2] ObjectStorage 신규 생성 

In [ ]:
# 스토리지 생성
res = api.create(
    stype="ObjectStorage",
    name="TestObject-01.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)
res 

In [ ]:
file = os.path.join(os.getcwd(), "01_Storages", "Create A Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.3] 생성한 스토리지 찾기

In [ ]:
# 스토리지 검색-1
res = api.search_by_name(
    name="TestObject-01.CreatedByRestAPI", 
    # workspace_id="dw-global-000000-default"
)
res 

In [ ]:
file = os.path.join(os.getcwd(), "01_Storages", "Search-1 the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.4] 스토리지 비우기 
- 샘플파일 아무거나 수동으로 업로드

In [ ]:
resource_uuid = "4407fe2c-1739-4391-9843-d120d7fe6cd7"

In [ ]:
res = api.clear(resource_uuid)
res 

- res.json() 없음 --> 결과는 플랫폼에서 직접 확인  

### [4.5] 스토리지 Export

In [ ]:
res = api.export_(resource_uuid)
res 

In [ ]:
file = os.path.join(os.getcwd(), "01_Storages", "Export the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.6] 스토리지 Import

In [ ]:
payload = {
    "name": "TestObject-05.ImportByRestAPI",
    "identifier": "TestObject-05.ImportByRestAPI",
    "update": True,
    "content": "ewogICJzdG9yYWdlIiA6IHsKICAgICJAY2xhc3MiIDogIk9iamVjdFN0b3JhZ2UiLAogICAgInJlc291cmNlSWQiIDogIlRlc3RPYmplY3QtMDMuQ3JlYXRlZEJ5UmVzdEFQSSIsCiAgICAibmFtZSIgOiAiVGVzdE9iamVjdC0wMy5DcmVhdGVkQnlSZXN0QVBJIiwKICAgICJkZXNjcmlwdGlvbiIgOiAi7YWM7Iqk7Yq4IO2bhCDrsJTroZwg7IKt7KCc7ZWgIOqygyIsCiAgICAia2luZCIgOiAiU3RvcmFnZSIsCiAgICAidHlwZSIgOiAiT2JqZWN0U3RvcmFnZSIsCiAgICAicmVzb3VyY2VQYXRoIiA6ICJTdG9yYWdlL1Rlc3RPYmplY3QtMDMuQ3JlYXRlZEJ5UmVzdEFQSS5qc29uIiwKICAgICJjb25maWciIDogewogICAgICAiYnVja2V0IiA6ICJ0ZXN0b2JqZWN0MDNjcmVhdGVkYnlyZXN0YXBpIiwKICAgICAgImRlbGVnYXRpb25zIiA6IFsgewogICAgICAgICJuYW1lIiA6ICJTZW1hbnRpY0dyYXBoSW5kZXguRE9TQWNjb3VudC4yIiwKICAgICAgICAicGVybWlzc2lvbiIgOiAicmVhZCIKICAgICAgfSwgewogICAgICAgICJuYW1lIiA6ICJTZW1hbnRpY0dyYXBoSW5kZXguRE9TQWNjb3VudC4yIiwKICAgICAgICAicGVybWlzc2lvbiIgOiAid3JpdGUiCiAgICAgIH0sIHsKICAgICAgICAibmFtZSIgOiAiUFBFUy5Eb3NBY2NvdW50IiwKICAgICAgICAicGVybWlzc2lvbiIgOiAiZXhlY3V0ZSIKICAgICAgfSwgewogICAgICAgICJuYW1lIiA6ICJEYXRhc2V0R292ZXJuYW5jZSIsCiAgICAgICAgInBlcm1pc3Npb24iIDogInJlYWQiCiAgICAgIH0gXQogICAgfQogIH0sCiAgInZlcnNpb24iIDogewogICAgInZhbHVlIiA6ICIxLjIiCiAgfQp9"
}

In [ ]:
from importlib import reload
import dserestapi
reload(dserestapi)
from dserestapi import Storages
api = Storages()
res = api.import_(payload)
res 

### [4.7] 생성한 스토리지 삭제

In [ ]:
# API를 이용해서 재검색
res = api.search_by_name(
    name="TestObject-01.CreatedByRestAPI"
)
resource_uuid = res.json()["cards"][0]["resourceUUID"]
resource_uuid

In [ ]:
res = api.delete(resource_uuid=resource_uuid)
res 

- res.json() --> 삭제 함수 응답에는 JSON 데이터가 없기 때문에 에러 발생

# **********************************************************************

# [5] 사용 예제 | Object Storage

### [5.1] Object Storage에 파일 업로드 

In [ ]:
# 신규 스토리지 재생성 
from dserestapi import Storages
api = Storages()
res = api.create(
    stype="ObjectStorage",
    name="TestObject-01.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)
res.json()

In [ ]:
# 생성한 스토리지의 resourceUUID 추출 
resource_uuid = res.json()["resourceUUID"]
# resource_uuid = "4407fe2c-1739-4391-9843-d120d7fe6cd7"
resource_uuid

#### [5.1.1] 파일 업로드 | 경로 미지정
- 업로드할 파일은 항상 파일 절대경로를 사용해야함

In [ ]:
from dserestapi import ObjectStorage
api = ObjectStorage()
uploading_file = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\CML Agent ID-PW.json"
res = api.upload(resourceUUID=resource_uuid, file=uploading_file, path=None)
res 

#### [5.1.2] 파일 업로드 | 경로 강제 지정

In [ ]:
api = ObjectStorage()
uploading_file = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\CML Agent ID-PW.json"
res = api.upload(resourceUUID=resource_uuid, file=uploading_file, path="Manual_Path_01/")
res 

- JSON 데이터 없음

### [5.2] Object Storage에 디렉토리 내의 모든 파일들을 한번에 업로드 

In [ ]:
# 업로드할 디렉토리 확인
import os 
top_dir = r"E:\__ENGINEER_DRIVE__\__DATASET__\__Dummy Data__"
uploading_files = []
for root, dirs, files in os.walk(top_dir):
    for file in files:
        uploading_files.append(os.path.join(root, file))

uploading_files

#### [5.2.1] 멀티 파일 업로드 | 자동 경로 설정

In [ ]:
# 멀티 파일 업로드 | 자동 경로 설정
import pprint 
pp = pprint.PrettyPrinter(indent=2)
api = ObjectStorage()
res_li = api.upload_files(resourceUUID=resource_uuid, files=uploading_files, path=None)
pp.pprint(res_li)

#### [5.2.2] 멀티 파일 업로드 | 수동 경로 설정

In [ ]:
# 멀티 파일 업로드 | 수동 경로 설정
api = ObjectStorage()
res_li = api.upload_files(resourceUUID=resource_uuid, files=uploading_files, path="Multi/")
pp.pprint(res_li)

- res.json() 없음 --> 업로드 결과는 플랫폼에서 직접 확인  

### [5.3] 스토리지 EXPORT 

In [ ]:
api = Storages()
res = api.export_(resource_uuid=resource_uuid)
res 

In [ ]:
file = os.path.join(os.getcwd(), "01_Storages", "Export Object Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

# **********************************************************************

# [6] 사용 예제 | Semantic Graph Index Unit

### [6.1] SGI Storage 신규 생성시 데이터 모델링을 동시에 수행 

In [ ]:
# 데이터 모델링 
data_modeling_config = {
    "datamodel": {
        "classes": [
            {
                "name": "DummyData_01",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            },
            {
                "name": "DummyData_02",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            }
        ]
    }
}

In [ ]:
from dserestapi import Storages
# 스토리지 생성
api = Storages()
res = api.create(
    stype="IndexUnit",
    name="TestSGI_01_CreatedByRestAPI",
    description="SGI 데이터 모델링 테스트 후 삭제할 것",
    config=data_modeling_config
)
res.json() 

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Create A SGI.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.2] 데이터 모델 스키마에 맞는 데이터 생성 후 스키마에 맞는 데이터 타입인지 정합성 체크 

In [ ]:
resource_uuid = res.json()["resourceUUID"]
# resource_uuid = "b1176f72-a3e2-4afc-9b02-543859e0cdf2"
resource_uuid

In [ ]:
# from dserestapi import SemanticGraphIndex
# sgi = SemanticGraphIndex()
# res = sgi.class_count(resourceUUID=resource_uuid, pkg_name="Rawdata", class_name_li=["DummyData_01"])
# with open(os.path.join(os.getcwd(), "Class Count.json"), "w", encoding="utf-8") as f:
#     json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [ ]:
# 데이터 생성 
data = [
    {
        "uri": "Rawdata.DummyData_01.0001",
        "class": "Rawdata.DummyData_01",
        "seq": 1,
        "title": "This is a title-1",
        "_text": "bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
    {
        "uri": "Rawdata.DummyData_01.0002",
        "class": "Rawdata.DummyData_01",
        "seq": 2,
        "title": "This is a title-2",
        "_text": "(2) bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
]

In [ ]:
# 저장할 데이터의 정합성 체크 
from dserestapi import SemanticGraphIndex
sgi = SemanticGraphIndex()
res = sgi.validateItemsEvent(resourceUUID=resource_uuid, action="AddOrReplaceItem", data=data)
res.json()

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Validation Items.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.3] 생성한 SGI의 타겟 데이터 모델에 데이터를 저장

In [ ]:
res = sgi.ingest(resourceUUID=resource_uuid, data=data)
res.json()

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Ingest Data.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

- 데이터가 저장되었는지 플랫폼에서 직접확인

### [6.4] 데이터 저장 결과 알림 받기 

In [ ]:
res = sgi.notification(resourceUUID=resource_uuid)
res.json()

### [6.5] 저장한 데이터의 object 별 uri 가져오기

In [ ]:
res = sgi.get_uri(resourceUUID=resource_uuid)
res.json()

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "SGI URIs.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

# **********************************************************************

# THE END.